In [0]:
from pyspark.sql.functions import col, explode, struct, array, lit
from pyspark.sql.types import *
import utils

xml_tag_to_extract="PeptideEvidence"

catalog = dbutils.widgets.get("catalog")
storage_account= dbutils.widgets.get("storage_account_name")
container=dbutils.widgets.get("container")
path_to_monitor=dbutils.widgets.get("path_to_monitor")
schema = dbutils.widgets.get("schema")

target_table= f"{catalog}.{schema}.{xml_tag_to_extract.lower()}_silver"

source_table = f"{catalog}.{schema}.{xml_tag_to_extract.lower()}_bronze"

checkpoint_location = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoint/silver/{xml_tag_to_extract.lower()}"


table_schema = StructType([
    StructField("_dBSequence_ref", StringType(), True),
    StructField("_end", LongType(), True),
    StructField("_id", StringType(), True),
    StructField("_isDecoy", BooleanType(), True),
    StructField("_peptide_ref", StringType(), True),
    StructField("_post", StringType(), True),
    StructField("_pre", StringType(), True),
    StructField("_start", LongType(), True),
    StructField("source_file", StringType(), True),
    StructField("file_size", LongType(), True),
    ])



utils.create_table_from_schema(spark, table_schema,target_table)

In [0]:

result = (
    spark
    .readStream
    .table(source_table)
    .filter(col("_isDecoy")==lit("false"))
    .writeStream
    .format("delta")      
    .option("mode","append")\
    .trigger(availableNow=True)\
    .option("checkpointLocation",checkpoint_location)\
    .table(target_table)
)